# Lab 4.2 - 高效數據篩選
## Notebook 01: 數據準備與環境配置

**學習目標**:
1. 加載 Alpaca 指令數據集 (52K 樣本)
2. 進行統計分析 (長度、領域分布)
3. 準備 Sentence-BERT 模型
4. 設定篩選目標 (30% 保留率)
5. 可視化數據分布

**預計時間**: 30-45 分鐘

---

## 1. 環境檢查

首先驗證必要的依賴是否安裝正確。

In [ ]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("=" * 60)
print("環境檢查")
print("=" * 60)

# Python 版本
print(f"Python 版本: {sys.version.split()[0]}")

# 檢查核心依賴
dependencies = {
    'numpy': 'numpy',
    'pandas': 'pandas',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'sentence-transformers': 'sentence_transformers',
    'scikit-learn': 'sklearn',
    'datasets': 'datasets',
    'tqdm': 'tqdm'
}

missing = []
for name, module in dependencies.items():
    try:
        __import__(module)
        print(f"✅ {name}")
    except ImportError:
        print(f"❌ {name} (未安裝)")
        missing.append(name)

if missing:
    print(f"\n⚠️  缺少依賴: {', '.join(missing)}")
    print(f"安裝指令: pip install {' '.join(missing)}")
else:
    print(f"\n✅ 所有依賴已安裝")

print("=" * 60)

## 2. 導入依賴

導入所需的 Python 套件。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tqdm import tqdm
import json
from datetime import datetime
from collections import Counter

# 設定可視化風格
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# 設定隨機種子
np.random.seed(42)

print("✅ 依賴導入完成")

## 3. 建立目錄結構

建立用於存放數據、分析結果和輸出的目錄。

In [ ]:
# 建立目錄結構
directories = {
    'data': Path('./data'),
    'analysis': Path('./analysis'),
    'results': Path('./results'),
    'validation': Path('./validation'),
    'pipeline': Path('./pipeline')
}

for name, path in directories.items():
    path.mkdir(exist_ok=True)
    print(f"✅ 建立目錄: {path}")

print(f"\n目錄結構建立完成")

## 4. 加載 Alpaca 數據集

加載 Stanford Alpaca 數據集 (52K 樣本)。

In [ ]:
print("📥 加載 Alpaca 數據集...")
print("=" * 60)

try:
    # 從 HuggingFace 加載 Alpaca 數據集
    dataset = load_dataset("tatsu-lab/alpaca", split="train")
    
    # 轉換為 list of dicts
    raw_data = []
    for item in dataset:
        sample = {
            'instruction': item['instruction'],
            'input': item.get('input', ''),
            'output': item['output']
        }
        raw_data.append(sample)
    
    print(f"✅ 數據集加載成功")
    print(f"   總樣本數: {len(raw_data):,}")
    
    # 保存原始數據
    raw_data_path = directories['data'] / 'alpaca_raw.json'
    with open(raw_data_path, 'w', encoding='utf-8') as f:
        json.dump(raw_data, f, indent=2, ensure_ascii=False)
    
    print(f"   已保存至: {raw_data_path}")
    
except Exception as e:
    print(f"❌ 錯誤: {e}")
    print("\n💡 提示: 嘗試使用本地數據或其他數據集")
    raw_data = []

print("=" * 60)

### 查看樣本範例

顯示幾個數據樣本以了解數據格式。

In [ ]:
if raw_data:
    print("\n📄 數據樣本:")
    print("=" * 60)
    
    for i in range(min(3, len(raw_data))):
        sample = raw_data[i]
        print(f"\n樣本 {i+1}:")
        print(f"  指令: {sample['instruction']}")
        if sample['input']:
            print(f"  輸入: {sample['input']}")
        print(f"  輸出: {sample['output'][:100]}..." if len(sample['output']) > 100 else f"  輸出: {sample['output']}")
    
    print("\n" + "=" * 60)
else:
    print("⚠️  數據未加載,跳過樣本顯示")

## 5. 數據統計分析

分析數據集的基本統計特徵。

In [ ]:
if raw_data:
    print("📊 數據統計分析")
    print("=" * 60)
    
    # 計算文本長度 (字符數)
    instruction_lengths = [len(s['instruction']) for s in raw_data]
    input_lengths = [len(s['input']) for s in raw_data]
    output_lengths = [len(s['output']) for s in raw_data]
    
    # 計算詞數 (簡單分割)
    instruction_words = [len(s['instruction'].split()) for s in raw_data]
    output_words = [len(s['output'].split()) for s in raw_data]
    
    stats = {
        '總樣本數': len(raw_data),
        '平均指令長度 (字符)': np.mean(instruction_lengths),
        '平均輸出長度 (字符)': np.mean(output_lengths),
        '平均指令詞數': np.mean(instruction_words),
        '平均輸出詞數': np.mean(output_words),
        '有輸入的樣本數': sum(1 for s in raw_data if s['input']),
        '無輸入的樣本數': sum(1 for s in raw_data if not s['input'])
    }
    
    for key, value in stats.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value:,}")
    
    # 保存統計信息
    stats_path = directories['analysis'] / 'data_statistics.json'
    with open(stats_path, 'w', encoding='utf-8') as f:
        json.dump(stats, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ 統計信息已保存至: {stats_path}")
    print("=" * 60)
else:
    print("⚠️  數據未加載,跳過統計分析")

## 6. 可視化數據分布

生成數據分布的可視化圖表。

In [ ]:
if raw_data:
    print("📈 生成數據分布圖表...")
    
    # 建立子圖
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Alpaca 數據集分布分析', fontsize=16, fontweight='bold')
    
    # 1. 指令長度分布
    axes[0, 0].hist(instruction_words, bins=50, edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(np.mean(instruction_words), color='red', linestyle='--', 
                       label=f'均值: {np.mean(instruction_words):.1f}')
    axes[0, 0].set_xlabel('詞數')
    axes[0, 0].set_ylabel('樣本數')
    axes[0, 0].set_title('指令長度分布')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 輸出長度分布
    axes[0, 1].hist(output_words, bins=50, edgecolor='black', alpha=0.7, color='orange')
    axes[0, 1].axvline(np.mean(output_words), color='red', linestyle='--', 
                       label=f'均值: {np.mean(output_words):.1f}')
    axes[0, 1].set_xlabel('詞數')
    axes[0, 1].set_ylabel('樣本數')
    axes[0, 1].set_title('輸出長度分布')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 指令 vs 輸出長度散點圖
    sample_size = min(5000, len(raw_data))
    sample_indices = np.random.choice(len(raw_data), sample_size, replace=False)
    sample_instr = [instruction_words[i] for i in sample_indices]
    sample_out = [output_words[i] for i in sample_indices]
    
    axes[1, 0].scatter(sample_instr, sample_out, alpha=0.3, s=10)
    axes[1, 0].set_xlabel('指令長度 (詞數)')
    axes[1, 0].set_ylabel('輸出長度 (詞數)')
    axes[1, 0].set_title(f'指令與輸出長度關係 (樣本: {sample_size:,})')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 輸入數據統計
    has_input = sum(1 for s in raw_data if s['input'])
    no_input = len(raw_data) - has_input
    
    axes[1, 1].pie([has_input, no_input], 
                   labels=[f'有輸入\n({has_input:,})', f'無輸入\n({no_input:,})'],
                   autopct='%1.1f%%',
                   startangle=90,
                   colors=['#66b3ff', '#ff9999'])
    axes[1, 1].set_title('樣本輸入分布')
    
    plt.tight_layout()
    
    # 保存圖表
    fig_path = directories['analysis'] / 'data_distribution.png'
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"✅ 圖表已保存至: {fig_path}")
    
    plt.show()
else:
    print("⚠️  數據未加載,跳過可視化")

## 7. 準備 Sentence-BERT 模型

加載 Sentence-BERT 模型用於後續的 IFD 計算。

In [ ]:
print("📥 加載 Sentence-BERT 模型...")
print("=" * 60)

# 使用平衡質量與速度的模型
MODEL_NAME = 'sentence-transformers/all-mpnet-base-v2'

try:
    embedding_model = SentenceTransformer(MODEL_NAME)
    
    print(f"✅ 模型加載成功: {MODEL_NAME}")
    print(f"   最大序列長度: {embedding_model.max_seq_length}")
    print(f"   嵌入維度: {embedding_model.get_sentence_embedding_dimension()}")
    
    # 測試編碼
    test_text = "This is a test sentence."
    test_embedding = embedding_model.encode([test_text])
    print(f"   測試編碼: {test_embedding.shape}")
    
except Exception as e:
    print(f"❌ 錯誤: {e}")
    print("\n💡 提示: 可能需要網路連接下載模型")
    embedding_model = None

print("=" * 60)

## 8. 設定篩選目標

定義數據篩選的目標和參數。

In [ ]:
# 篩選配置
filtering_config = {
    'source_dataset': 'Alpaca',
    'total_samples': len(raw_data) if raw_data else 0,
    'target_retention_rate': 0.30,
    'target_samples': int(len(raw_data) * 0.30) if raw_data else 0,
    
    # IFD 參數
    'ifd_min_threshold': 0.3,
    'ifd_max_threshold': 0.9,
    
    # DEITA 權重
    'deita_alpha': 0.4,  # 複雜度權重
    'deita_beta': 0.4,   # 質量權重
    'deita_gamma': 0.2,  # 多樣性權重
    
    # 其他參數
    'embedding_model': MODEL_NAME,
    'batch_size': 64,
    'timestamp': datetime.now().isoformat()
}

print("🎯 篩選配置")
print("=" * 60)
print(json.dumps(filtering_config, indent=2, ensure_ascii=False))
print("=" * 60)

# 保存配置
config_path = directories['data'] / 'filtering_config.json'
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(filtering_config, f, indent=2, ensure_ascii=False)

print(f"\n✅ 配置已保存至: {config_path}")

## 9. 環境驗證總結

檢查所有設置是否正確完成。

In [ ]:
print("\n" + "=" * 60)
print("環境驗證總結")
print("=" * 60)

checks = [
    ("數據集加載", len(raw_data) > 0 if raw_data else False),
    ("統計分析完成", (directories['analysis'] / 'data_statistics.json').exists()),
    ("可視化生成", (directories['analysis'] / 'data_distribution.png').exists()),
    ("Sentence-BERT 模型", embedding_model is not None),
    ("篩選配置建立", (directories['data'] / 'filtering_config.json').exists())
]

for check_name, status in checks:
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {check_name}")

all_passed = all(status for _, status in checks)

print("\n" + "=" * 60)
if all_passed:
    print("✅ 所有檢查通過! 可以繼續進行篩選 (Notebook 02)")
    print(f"\n預期篩選結果: {filtering_config['total_samples']:,} → {filtering_config['target_samples']:,} 樣本 (保留 30%)")
else:
    print("⚠️  部分檢查未通過,請解決上述問題後再繼續")
print("=" * 60)

## 📝 總結

在本 notebook 中,我們完成了:

1. ✅ 環境檢查與依賴驗證
2. ✅ 加載 Alpaca 數據集 (52K 樣本)
3. ✅ 數據統計分析 (長度、分布)
4. ✅ 生成可視化圖表
5. ✅ 準備 Sentence-BERT 模型
6. ✅ 設定篩選配置 (30% 保留率)

### 關鍵觀察

- 平均指令長度: ~15 詞
- 平均輸出長度: ~78 詞
- 輸入分布: 約 40% 樣本有額外輸入

### 下一步

前往 **02-Filter.ipynb** 執行 IFD 和 DEITA 篩選流程。

---

**記得**:
- 篩選目標: 從 52K 樣本中篩選出 15.6K 高質量樣本
- IFD 閾值: 0.3 ≤ IFD ≤ 0.9
- DEITA 權重: 複雜度 40%, 質量 40%, 多樣性 20%
- 預期效果: 質量提升,訓練時間減少 70%